<a href="https://colab.research.google.com/github/dearaa/Inventory-Analysis-using-XGboost/blob/main/Astro_Dea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_gbq --upgrade
!pip install --upgrade google-cloud-bigquery


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Authenticate and create BigQuery client
client = bigquery.Client(project="astro-project-468008")

# ✅ Use the correct full table names from the public dataset
query = """
    WITH inventory AS (
      SELECT
        id AS inventory_item_id,
        product_id,
        created_at,
        sold_at,
        cost,
        product_category,
        product_name,
        product_brand,
        product_retail_price,
        product_department,
        product_sku,
        product_distribution_center_id
      FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
    ),

    order_item AS (
      SELECT
        id,
        order_id,
        user_id,
        product_id,
        inventory_item_id,
        status,
        shipped_at,
        delivered_at,
        returned_at,
        sale_price
      FROM `bigquery-public-data.thelook_ecommerce.order_items`
    ),

    orders AS (
      SELECT
        order_id,
        user_id,
        status,
        gender,
        created_at,
        returned_at,
        shipped_at,
        delivered_at,
        num_of_item
      FROM `bigquery-public-data.thelook_ecommerce.orders`
    ),

    dc AS (
      SELECT
        id,
        name,
        longitude,
        latitude
      FROM `bigquery-public-data.thelook_ecommerce.distribution_centers`
    ),

    product AS (
      SELECT
        id AS product_id,
        category,
        name,
        brand,
        cost,
        retail_price,
        department,
        distribution_center_id
      FROM `bigquery-public-data.thelook_ecommerce.products`
    ),

    base_demand AS (
      SELECT
        inv.product_id,
        case when pr.department = 'Men' then 1 else 0 end as product_men,
        case when pr.department = 'Women' then 1 else 0 end as product_women,
        inv.product_distribution_center_id AS distribution_center_id,
        date_trunc(o.created_at, month) AS order_month,
        COUNT(DISTINCT oi.id) AS total_orders,
        COUNT(oi.id) AS total_quantity,
        SUM(oi.sale_price) AS total_sales,
        AVG(oi.sale_price) AS avg_sale_price,
        MAX(pr.retail_price) AS max_retail_price,
        SAFE_DIVIDE(MAX(pr.retail_price) - AVG(oi.sale_price), MAX(pr.retail_price)) AS price_discount_pct,
        EXTRACT(MONTH FROM o.created_at) AS month_number,
        CASE WHEN EXTRACT(MONTH FROM o.created_at) IN (11, 12) THEN 1 ELSE 0 END AS is_holiday_season
      FROM order_item oi
      JOIN inventory inv
        ON oi.inventory_item_id = inv.inventory_item_id
      JOIN orders o
        ON oi.order_id = o.order_id
      LEFT JOIN product pr
        ON inv.product_id = pr.product_id
      left join dc
        ON dc.id = pr.distribution_center_id
      WHERE o.status = 'Complete'
      GROUP BY 1, 2, 3,4,5,13,12
    ),

    with_lag AS (
      SELECT *,
        LAG(total_quantity, 1) OVER (PARTITION BY product_id, distribution_center_id ORDER BY order_month) AS lag_1m_qty,
        SUM(total_quantity) OVER (
          PARTITION BY product_id, distribution_center_id
          ORDER BY order_month
          ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS rolling_3m_qty,
        AVG(total_quantity) OVER (
          PARTITION BY product_id, distribution_center_id
          ORDER BY order_month
          ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS rolling_3m_avg
      FROM base_demand
    ),

    latest_inventory AS (
      SELECT
        product_id,
        product_distribution_center_id AS distribution_center_id,
        COUNT(*) AS inventory_count
      FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
      WHERE sold_at IS NULL -- still available
      GROUP BY 1, 2
    )

    SELECT
      w.*,
      li.inventory_count
    FROM with_lag w
    LEFT JOIN latest_inventory li
      ON w.product_id = li.product_id
      AND w.distribution_center_id = li.distribution_center_id
    ORDER BY w.product_id, w.distribution_center_id, w.order_month


"""

# Run query and get results in DataFrame
df = client.query(query).to_dataframe()
df.head()


,product_id,product_men,product_women,distribution_center_id,order_month,total_orders,total_quantity,total_sales,avg_sale_price,max_retail_price,price_discount_pct,month_number,is_holiday_season,lag_1m_qty,rolling_3m_qty,rolling_3m_avg,inventory_count
0,1,0,1,1,2023-02-01 00:00:00+00:00,1,1,49.0,49.0,49.0,0.0,2,0,<NA>,1,1.0,7
1,1,0,1,1,2024-12-01 00:00:00+00:00,1,1,49.0,49.0,49.0,0.0,12,1,1,2,1.0,7
2,2,0,1,3,2025-07-01 00:00:00+00:00,1,1,69.5,69.5,69.5,0.0,7,0,<NA>,1,1.0,8
3,3,0,1,8,2025-05-01 00:00:00+00:00,1,1,69.5,69.5,69.5,0.0,5,0,<NA>,1,1.0,8
4,4,0,1,6,2021-09-01 00:00:00+00:00,1,1,108.0,108.0,108.0,0.0,9,0,<NA>,1,1.0,11


In [ ]:
# Template
!pip install pandas scikit-learn xgboost matplotlib seaborn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
# Feature Engineering

df['order_month'] = pd.to_datetime(df['order_month'])

# Sort
df = df.sort_values(by=['product_id', 'distribution_center_id', 'order_month'])

# Create lag features
df['lag_1m_stock'] = df.groupby(['product_id', 'distribution_center_id'])['inventory_count'].shift(1)
df['lag_2m_stock'] = df.groupby(['product_id', 'distribution_center_id'])['inventory_count'].shift(2)

# Create rolling features
df['rolling_3m_order_qty'] = df.groupby(['product_id', 'distribution_center_id'])['total_orders'].transform(lambda x: x.rolling(3, 1).mean())

# Target Variable: stock_qty next month
df['target_stock_qty_next_month'] = df.groupby(['product_id', 'distribution_center_id'])['inventory_count'].shift(-1)

# Drop null from shift
df = df.dropna()

df.head()


,product_id,product_men,product_women,distribution_center_id,order_month,total_orders,total_quantity,total_sales,avg_sale_price,max_retail_price,...,month_number,is_holiday_season,lag_1m_qty,rolling_3m_qty,rolling_3m_avg,inventory_count,lag_1m_stock,lag_2m_stock,rolling_3m_order_qty,target_stock_qty_next_month
6,4,0,1,6,2022-03-01 00:00:00+00:00,1,1,108.0,108.0,108.0,...,3,0,1,3,1.0,11,11,11,1.0,11
7,4,0,1,6,2022-05-01 00:00:00+00:00,1,1,108.0,108.0,108.0,...,5,0,1,3,1.0,11,11,11,1.0,11
19,12,0,1,3,2022-11-01 00:00:00+00:00,1,1,99.0,99.0,99.0,...,11,1,1,3,1.0,12,12,12,1.0,12
20,12,0,1,3,2023-04-01 00:00:00+00:00,1,1,99.0,99.0,99.0,...,4,0,1,3,1.0,12,12,12,1.0,12
21,12,0,1,3,2025-01-01 00:00:00+00:00,1,1,99.0,99.0,99.0,...,1,0,1,3,1.0,12,12,12,1.0,12


In [ ]:
# Modeling Preparation

# Features & Target
features = [
    'total_orders',
    'total_sales',
    'rolling_3m_qty',
    'rolling_3m_avg',
    'inventory_count',
    'rolling_3m_order_qty',
    'lag_1m_stock',
    'lag_2m_stock',
    'is_holiday_season',
    'month_number',
    'total_quantity',
    'product_men',
    'product_women',
    'avg_sale_price'

]

target = 'target_stock_qty_next_month'

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df[features], df[target], test_size=0.2, random_state=42
)


In [ ]:
# Model: Random Forest Regressor

# Initialize model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Evaluation Metrics:")
print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")
print(f"✅ R² Score: {r2:.2f}")


Evaluation Metrics:
✅ RMSE: 0.04
✅ MAE: 0.00
✅ R² Score: 1.00


In [ ]:
# Try model XGBoost and tuning the model

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# 2. Feature Engineering (same as before)
df['month'] = pd.to_datetime(df['order_month'])
df['month_num'] = df['order_month'].dt.month
df['year'] = df['order_month'].dt.year

# 3. One-hot encode categorical variables
categorical_features = ['product_id']
df_encoded = pd.get_dummies(df, columns=categorical_features)

# 4. Select features and target

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# 6. XGBoost Regressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# 7. Hyperparameter Tuning
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5]
}

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# 8. Predict and Evaluate
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("📊 Evaluation Metrics (XGBoost + Tuning):")
print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")
print(f"✅ R² Score: {r2:.2f}")
print("🔍 Best Parameters:", random_search.best_params_)


Fitting 3 folds for each of 20 candidates, totalling 60 fits
📊 Evaluation Metrics (XGBoost + Tuning):
✅ RMSE: 0.04
✅ MAE: 0.00
✅ R² Score: 1.00
🔍 Best Parameters: {'subsample': 1.0, 'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.8}


In [ ]:
print(df.dtypes)


product_id                                   Int64
product_men                                  Int64
product_women                                Int64
distribution_center_id                       Int64
order_month                    datetime64[us, UTC]
total_orders                                 Int64
total_quantity                               Int64
total_sales                                float64
avg_sale_price                             float64
max_retail_price                           float64
price_discount_pct                         float64
month_number                                 Int64
is_holiday_season                            Int64
lag_1m_qty                                   Int64
rolling_3m_qty                               Int64
rolling_3m_avg                             float64
inventory_count                              Int64
lag_1m_stock                                 Int64
lag_2m_stock                                 Int64
rolling_3m_order_qty           

In [ ]:
# Handling Multicollinearity and eliminated the features

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. Select features and target
features = [
    'product_id', 'product_men', 'product_women',
    'distribution_center_id', 'month_number', 'is_holiday_season',
    'total_orders', 'total_quantity', 'total_sales',
    'avg_sale_price', 'max_retail_price', 'price_discount_pct',
    'lag_1m_qty', 'rolling_3m_qty', 'rolling_3m_avg',
    'lag_1m_stock', 'lag_2m_stock', 'rolling_3m_order_qty'
]

target = 'target_stock_qty_next_month'

X = df[features]
y = df[target]

# 2. Confirm all numeric
assert X.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x)).all(), "Not all features are numeric!"
assert pd.api.types.is_numeric_dtype(y), "Target is not numeric!"

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create DataFrame for VIF
vif_df = pd.DataFrame()
vif_df['feature'] = X.columns
vif_df['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

print(vif_df.sort_values(by='VIF', ascending=False))


                   feature         VIF
1              product_men         inf
16            lag_2m_stock         inf
2            product_women         inf
7           total_quantity         inf
6             total_orders         inf
14          rolling_3m_avg         inf
13          rolling_3m_qty         inf
10        max_retail_price         inf
9           avg_sale_price         inf
17    rolling_3m_order_qty         inf
15            lag_1m_stock         inf
8              total_sales  166.246619
0               product_id    4.256636
12              lag_1m_qty    2.194561
5        is_holiday_season    1.854056
4             month_number    1.851046
3   distribution_center_id    1.010314
11      price_discount_pct         NaN


In [ ]:
# Cleaned features with reduced multicollinearity
features = [
    'distribution_center_id',
    'product_men'
    'month_number',
    'is_holiday_season',
    'total_orders',
    'lag_1m_qty',
    'rolling_3m_qty',
    'avg_sale_price'
]

target = 'target_stock_qty_next_month'

X = df[features]
y = df[target]

# Train/test split and XGBoost as before
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("✅ RMSE:", round(rmse, 2))
print("✅ MAE:", round(mae, 2))
print("✅ R² Score:", round(r2, 2))


✅ RMSE: 4.44
✅ MAE: 3.52
✅ R² Score: 0.11


In [ ]:
# Tuning The Model

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("✅ Best Parameters:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")
print(f"✅ R² Score: {r2:.2f}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
✅ Best Parameters:
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 1.0}
✅ RMSE: 4.35
✅ MAE: 3.41
✅ R² Score: 0.14


In [ ]:
# Model: Random Forest Regressor

# Initialize model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Evaluation Metrics:")
print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ MAE: {mae:.2f}")
print(f"✅ R² Score: {r2:.2f}")


Evaluation Metrics:
✅ RMSE: 4.24
✅ MAE: 3.32
✅ R² Score: 0.19


In [ ]:
# Coba Log transform

y_log = np.log1p(y)

In [ ]:
# Train/test split and XGBoost as before
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

model = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("✅ RMSE:", round(rmse, 2))
print("✅ MAE:", round(mae, 2))
print("✅ R² Score:", round(r2, 2))


✅ RMSE: 0.27
✅ MAE: 0.21
✅ R² Score: 0.08


In [ ]:
# Coba pake target binning

import numpy as np

# Create bins based on percentiles
df['target_bin'] = pd.qcut(
    df['target_stock_qty_next_month'],
    q=[0, 0.25, 0.75, 1.0],
    labels=['Low', 'Medium', 'High']
)

df['target_bin'] = df['target_bin'].astype(str)
df['target_bin'].value_counts()

,count
target_bin,
Medium,1223
Low,783
High,555


In [ ]:
#lightGBM pake target binning

from sklearn.model_selection import train_test_split

X = df[features]
y = df['target_bin']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

clf = LGBMClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000204 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 544
[LightGBM] [Info] Number of data points in the train set: 2048, number of used features: 7
[LightGBM] [Info] Start training from score -1.528794
[LightGBM] [Info] Start training from score -1.185269
[LightGBM] [Info] Start training from score -0.739109
[[ 44  19  48]
 [ 15  60  82]
 [ 32  44 169]]
              precision    recall  f1-score   support

        High       0.48      0.40      0.44       111
         Low       0.49      0.38      0.43       157
      Medium       0.57      0.69      0.62       245

    accuracy                           0.53       513
   macro avg       0.51      0.49      0.50       513
weighted avg       0.52      0.53      0.52       513



In [ ]:
#xgboost pake target binning

from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

X = df[features]
y = df['target_bin']

from sklearn.preprocessing import LabelEncoder

# 1. Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Cek hasil
print(le.classes_)  # ['High' 'Low' 'Medium'] -> nanti diubah ke 0,1,2

# 2. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 3. Build model
xgb_clf = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# 4. Fit model
xgb_clf.fit(X_train, y_train)

# 5. Predict
y_pred = xgb_clf.predict(X_test)

# 6. Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


['High' 'Low' 'Medium']
[[ 43  19  51]
 [ 18  52  81]
 [ 20  60 169]]
              precision    recall  f1-score   support

        High       0.53      0.38      0.44       113
         Low       0.40      0.34      0.37       151
      Medium       0.56      0.68      0.61       249

    accuracy                           0.51       513
   macro avg       0.50      0.47      0.48       513
weighted avg       0.51      0.51      0.50       513

